In [5]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from utils import make_ftranslate, get_files, save_ftranslate
pd.options.io.excel.xlsx.writer = 'xlsxwriter'

def get_keys(d, value):
    return [k for k, v in d.items() if v == value]

save_ftranslate()
# china_dataset = pd.read_csv(r'C:\datas\企业信息数据\中国上市制造企业汇总表(简版).txt', index_col=0, dtype={'股票代码': "object"})
# Js_codes = set(china_dataset['股票代码'].to_list())
china_dataset = pd.DataFrame(columns=['Stkcd'])
financials, indicator_dict = make_ftranslate()
# china_dataset.rename(columns={'股票代码': get_keys(indicator_dict, '股票代码')[0], '省份':'上市省份'}, inplace=True)
files = get_files(r'C:\datas\企业绩效数据\财务绩效数据')[0]

for file in files:
    dataset = pd.read_csv(file, sep='\t', header=0, dtype={'Stkcd': 'object'}, parse_dates=['Accper'])
    names = {item for item in dataset.columns.values.tolist() if item in financials}
    if 'Typrep' in dataset.columns:
        dataset = dataset[(dataset.Typrep == "A") & (dataset.Accper.dt.month == 12)& (dataset.Accper.dt.year > 2006)][list(names)]
    else:
        dataset = dataset[(dataset.Accper.dt.month == 12) & (dataset.Accper.dt.year > 2006)][list(names)]
    
    if 'Accper' not in china_dataset.columns:
        china_dataset = pd.merge(china_dataset, dataset, on=['Stkcd'], how='outer')
    else:
        china_dataset = pd.merge(china_dataset, dataset, on=['Stkcd', 'Accper'], how='outer')
    financials -= names
    financials.update(['Stkcd', 'Accper'])
china_dataset

,Stkcd,F012501B,F011601A,Indnme,Indcd,ShortName,Typrep,F010201A,F011701A,Source,...,F040902B,F040802B,F041802B,F040502B,F041202B,F040202B,F041002B,F041702B,F070201B,F070101B
0,000001,0.827344,27.105771,货币金融服务,J66,深发展A,A,NaN,26.105771,0.0,...,NaN,NaN,1.049990,NaN,NaN,NaN,0.390655,0.033561,NaN,NaN
1,000001,0.942320,28.927885,货币金融服务,J66,深发展A,A,NaN,27.927885,0.0,...,NaN,NaN,0.922453,NaN,NaN,0.924629,0.400805,0.032802,NaN,NaN
2,000001,0.886752,28.716281,货币金融服务,J66,深发展A,A,NaN,27.716281,0.0,...,NaN,0.000000,0.775357,NaN,NaN,0.568132,0.311318,0.026912,NaN,NaN
3,000001,0.929486,21.905454,货币金融服务,J66,深发展A,A,NaN,20.905454,0.0,...,NaN,NaN,0.654472,NaN,NaN,0.520823,0.285904,0.026710,NaN,NaN
4,000001,0.938057,16.690995,货币金融服务,J66,深发展A,A,NaN,15.690995,0.0,...,NaN,NaN,0.535786,NaN,NaN,0.268642,0.427584,0.029301,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60191,900957,0.359100,2.230210,房地产业,K70,凌云B股,A,2.100740,1.230210,0.0,...,1.054394,12.164382,0.242442,NaN,0.529933,0.918787,2.360203,0.104729,1.644900,2.242781
60192,900957,0.365438,2.134160,房地产业,K70,凌云B股,A,1.964754,1.134160,0.0,...,0.953994,16.174323,0.226244,NaN,0.483113,0.656185,2.992630,0.103740,1.670870,2.229302
60193,900957,0.377791,2.022270,房地产业,K70,凌云B股,A,2.991174,1.022270,0.0,...,0.714718,19.850004,0.203784,NaN,0.415326,0.515480,3.883610,0.098117,1.742779,2.299762
60194,900957,0.334505,1.944507,房地产业,K70,凌云B股,A,2.615010,0.944507,0.0,...,0.662935,21.101687,0.220605,NaN,0.424187,0.477495,13.885294,0.111285,1.658589,1.947996


In [8]:
# 财务报表合并
dataset_ = china_dataset.copy()
dataset_['Accper'] = dataset_['Accper'].dt.year
name_dic = {name:indicator_dict[name] for name in dataset_.columns.values.tolist()}
dataset_.rename(columns=name_dic, inplace=True)
dataset_.rename(columns={'统计截止日期' : '截止年份'}, inplace=True)
dataset_.dropna(subset=['股票简称'], inplace=True)
dataset_.drop(['报表类型编码','公告来源','行业名称','股票简称'], axis=1, inplace=True) # '行业名称', 
dataset_.sort_values(['股票代码', '截止年份'], inplace=True)
dataset_.to_csv(r'./datas/全国上市企业财务绩效汇总表.csv', index=None,encoding='utf8')
dataset_

,股票代码,负债与权益市价比率,权益乘数,行业代码,速动比率,产权比率,流动比率,资产负债率,截止年份,经营活动产生的现金流量净额／负债合计,...,营运资金(资本)周转率B,应付账款周转率B,股东权益周转率B,存货周转率B,流动资产周转率B,应收账款周转率B,现金及现金等价物周转率B,总资产周转率B,经营杠杆,财务杠杆
0,000001,0.827344,27.105771,J66,NaN,26.105771,NaN,0.963107,2007,0.057448,...,NaN,NaN,1.049990,NaN,NaN,NaN,0.390655,0.033561,NaN,NaN
1,000001,0.942320,28.927885,J66,NaN,27.927885,NaN,0.965431,2008,0.061042,...,NaN,NaN,0.922453,NaN,NaN,0.924629,0.400805,0.032802,NaN,NaN
2,000001,0.886752,28.716281,J66,NaN,27.716281,NaN,0.965177,2009,0.062793,...,NaN,0.000000,0.775357,NaN,NaN,0.568132,0.311318,0.026912,NaN,NaN
3,000001,0.929486,21.905454,J66,NaN,20.905454,NaN,0.954349,2010,0.034481,...,NaN,NaN,0.654472,NaN,NaN,0.520823,0.285904,0.026710,NaN,NaN
4,000001,0.938057,16.690995,J66,NaN,15.690995,NaN,0.940087,2011,-0.015387,...,NaN,NaN,0.535786,NaN,NaN,0.268642,0.427584,0.029301,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60191,900957,0.359100,2.230210,K70,2.100740,1.230210,2.100740,0.551612,2018,0.069444,...,1.054394,12.164382,0.242442,NaN,0.529933,0.918787,2.360203,0.104729,1.644900,2.242781
60192,900957,0.365438,2.134160,K70,1.964754,1.134160,1.964754,0.531432,2019,0.081498,...,0.953994,16.174323,0.226244,NaN,0.483113,0.656185,2.992630,0.103740,1.670870,2.229302
60193,900957,0.377791,2.022270,K70,2.991174,1.022270,2.991174,0.505506,2020,0.106316,...,0.714718,19.850004,0.203784,NaN,0.415326,0.515480,3.883610,0.098117,1.742779,2.299762
60194,900957,0.334505,1.944507,K70,2.615010,0.944507,2.615010,0.485731,2021,0.042833,...,0.662935,21.101687,0.220605,NaN,0.424187,0.477495,13.885294,0.111285,1.658589,1.947996
